In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy as np
from pycuda import gpuarray
from pycuda.compiler import SourceModule

Each kernel is unique field comnbination  
Up to 65536 combinations

In [2]:
ker = SourceModule("""
__global__ void combination_check(float *outvec)
{
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int a, c, s;
    int b = 0;
    int ones = 0;
    for(s=0;s<16;s++)
    {                
        a = i>>(16-s);        
        c = a - b;
        b = b+a;
        if (c == 1) ones += 1;
    }   
    if (ones==8) outvec[i] = 1;
}
""")

In [3]:
combination_check_gpu = ker.get_function("combination_check")

combinations = np.zeros(256*256).astype(np.float32)
combinations_gpu_in = gpuarray.to_gpu(combinations)
combinations_gpu_out = gpuarray.empty_like(combinations_gpu_in)

%time combination_check_gpu( combinations_gpu_out,  block=(256,1,1), grid=(256,1,1))

CPU times: user 631 µs, sys: 383 µs, total: 1.01 ms
Wall time: 715 µs


In [4]:
out_combinations = combinations_gpu_out.get()

In [5]:
np.count_nonzero(out_combinations)

256